# 1 Import library¶

In [212]:
import findspark
findspark.init('/home/system/spark-3.1.1-bin-hadoop3.2')
import pyspark
import pandas as pd
from pyspark.sql import SparkSession 

In [213]:
import pyarrow
import fastparquet

In [214]:
from pyspark.sql.functions import lower

In [215]:
from pyspark.sql.functions import col, asc, desc

# 2  task1
Сгенерировать DataFrame из трёх колонок (row_id, discipline, season) - олимпийские дисциплины по сезонам.

row_id - число порядкового номера строки;
discipline - наименование олимпиский дисциплины на английском (полностью маленькими буквами);
season - сезон дисциплины (summer / winter);
*Укажите не мнее чем по 5 дисциплин для каждого сезона.

Сохраните DataFrame в csv-файл, разделитель колонок табуляция, первая строка должна содержать название колонок.

Данные должны быть сохранены в виде 1 csv-файла а не множества маленьких.

In [247]:
spark = (SparkSession
 .builder
 .appName('pyspark_task1')
 .getOrCreate())

In [248]:
rows = [
(1,"skiing", "winter"),(2,"snowbording", "winter"),
(3,"figure_skating", "winter"), (4,"bobsleigh", "winter"),
(5,"ice_hockey", "winter"), (6,"biathlon", "winter"),
(7,"athletics", "summer"), (8,"swimming", "summer"), 
(9,"skateboarding", "summer"), (10,"surfing", "summer"),
(11,"gymnastics", "summer"), (12,"rowing", "summer")
]

In [249]:
schema = "row_id BIGINT, discipline STRING, season STRING"
olimpic_games_DF = spark.createDataFrame(rows, schema)
olimpic_games_DF.show()

+------+--------------+------+
|row_id|    discipline|season|
+------+--------------+------+
|     1|        skiing|winter|
|     2|   snowbording|winter|
|     3|figure_skating|winter|
|     4|     bobsleigh|winter|
|     5|    ice_hockey|winter|
|     6|      biathlon|winter|
|     7|     athletics|summer|
|     8|      swimming|summer|
|     9| skateboarding|summer|
|    10|       surfing|summer|
|    11|    gymnastics|summer|
|    12|        rowing|summer|
+------+--------------+------+



In [250]:
#сохранение через spark to csv
output_path = "home/olimpic_games.csv"
single_part_olimpic_games_DF = olimpic_games_DF.coalesce(1) #reduce amount portition
single_part_olimpic_games_DF.write.csv(output_path, header=True, mode="overwrite", sep="\t")

# 3 task 2
2) Прочитайте исходный файл "Athletes.csv".

Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.

Результат сохраните в формате parquet.

In [251]:
input_file = "home/Athletes.csv"
df = spark.read.csv(input_file, header = True, sep = ';')
df.cache()
df.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

23/08/31 09:15:23 WARN CacheManager: Asked to cache already cached data.


In [252]:
#see col, type data
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [253]:
#show all distinct Discipline
df.select(col('Discipline')).distinct().show(df.count(), truncate=False)

+---------------------+
|Discipline           |
+---------------------+
|Tennis               |
|Boxing               |
|Marathon Swimming    |
|Golf                 |
|Rowing               |
|Baseball/Softball    |
|Judo                 |
|Sailing              |
|Swimming             |
|Cycling BMX Freestyle|
|Basketball           |
|Handball             |
|Rhythmic Gymnastics  |
|Karate               |
|Triathlon            |
|Badminton            |
|Canoe Sprint         |
|Athletics            |
|Cycling Track        |
|Beach Volleyball     |
|Skateboarding        |
|Equestrian           |
|Shooting             |
|Diving               |
|Canoe Slalom         |
|Rugby Sevens         |
|Artistic Gymnastics  |
|Cycling Mountain Bike|
|3x3 Basketball       |
|Cycling Road         |
|Wrestling            |
|Table Tennis         |
|Taekwondo            |
|Water Polo           |
|Weightlifting        |
|Surfing              |
|Sport Climbing       |
|Cycling BMX Racing   |
|Trampoline Gymn

In [101]:
#выборка sparksql

In [254]:
df.createOrReplaceTempView("athletes")
athletes_DF = spark.sql("""
select Discipline,  COUNT(*) as Count from athletes
group by Discipline
""")
athletes_DF.orderBy("Count", ascending=False).show()

+-----------------+-----+
|       Discipline|Count|
+-----------------+-----+
|        Athletics| 2068|
|         Swimming|  743|
|         Football|  567|
|           Rowing|  496|
|           Hockey|  406|
|             Judo|  373|
|         Handball|  343|
|         Shooting|  342|
|          Sailing|  336|
|     Rugby Sevens|  283|
|       Basketball|  280|
|        Wrestling|  279|
|       Volleyball|  274|
|           Boxing|  270|
|       Water Polo|  269|
|          Fencing|  249|
|       Equestrian|  237|
|     Canoe Sprint|  236|
|Baseball/Softball|  220|
|    Cycling Track|  208|
+-----------------+-----+
only showing top 20 rows



In [105]:
#выборка spark -  считаю в данном случае уместнее spark использовать

In [255]:
aggregated_data = df.groupBy("Discipline").count().orderBy(col("count").desc())
aggregated_data.show()

+-----------------+-----+
|       Discipline|count|
+-----------------+-----+
|        Athletics| 2068|
|         Swimming|  743|
|         Football|  567|
|           Rowing|  496|
|           Hockey|  406|
|             Judo|  373|
|         Handball|  343|
|         Shooting|  342|
|          Sailing|  336|
|     Rugby Sevens|  283|
|       Basketball|  280|
|        Wrestling|  279|
|       Volleyball|  274|
|           Boxing|  270|
|       Water Polo|  269|
|          Fencing|  249|
|       Equestrian|  237|
|     Canoe Sprint|  236|
|Baseball/Softball|  220|
|    Cycling Track|  208|
+-----------------+-----+
only showing top 20 rows



In [256]:
output_path1 = "home/athletes_DF.parquet"
pandas_df = athletes_DF.toPandas() #тут преобразую в пандас дата фрейм для сохранения (небольшие дата сеты фреймы так ок)
pandas_df.to_parquet(path=output_path1, compression='snappy', index=False) #в задании не было упорядочивания, поэтому сохраню так
pd.read_parquet(output_path1).head(5) #проверка

,Discipline,Count
0,Tennis,178
1,Boxing,270
2,Marathon Swimming,49
3,Golf,115
4,Rowing,496


# 4 task3
Прочитайте исходный файл "Athletes.csv".

Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.

Получившийся результат нужно объединить с сгенерированным вами DataFrame из 1-го задания и в итоге вывести количество участников, только по тем дисциплинам, что есть в вашем сгенерированном DataFrame.

Результат сохраните в формате parquet.

In [257]:
merged_df = olimpic_games_DF.join(athletes_DF, olimpic_games_DF.discipline == lower(athletes_DF.Discipline), 'inner').select('athletes.Discipline', 'Count')
merged_df.show()

+-------------+-----+
|   Discipline|Count|
+-------------+-----+
|    Athletics| 2068|
|     Swimming|  743|
|Skateboarding|   77|
|      Surfing|   38|
|       Rowing|  496|
+-------------+-----+



In [258]:
#сохранение через пандас + проверка
output_path2 = "home/pandas_df_union.parquet"
pandas_df_union = merged_df.toPandas()
pandas_df_union.to_parquet(path=output_path2, compression='snappy', index=False)
pd.read_parquet(output_path2)

,Discipline,Count
0,Athletics,2068
1,Swimming,743
2,Skateboarding,77
3,Surfing,38
4,Rowing,496


In [259]:
#сохранение через spark + проверка
output_path3 = "home/single_part_merge_DF.parquet"
single_part_merge = merged_df.coalesce(1)
single_part_merge.write.parquet(output_path3, mode="overwrite")
df_single_part_merge_DF = spark.read.parquet(output_path3)
df_single_part_merge_DF.show()

+-------------+-----+
|   Discipline|Count|
+-------------+-----+
|    Athletics| 2068|
|     Swimming|  743|
|Skateboarding|   77|
|      Surfing|   38|
|       Rowing|  496|
+-------------+-----+



In [260]:
spark.stop()